# 日本語テキストの前処理

### ソフトウェアのインストール

spaCy
- https://spacy.io/usage
- https://anaconda.org/anaconda/spacy
```
conda install spacy
python -m spacy download ja_core_news_lg
```

### 処理内容

- 前処理
- ベクトル化

### spaCy の日本語モデル

- 以下の spaCy の日本語モデルのロードでエラーが出る場合はノートブックをリスタート

In [1]:
import spacy

# 別の日本語モデルのダウンロード方法
# python -m spacy download ja_core_news_lg がうまくいかない場合はコメントアウト
# spacy.cli.download("ja_core_news_lg")

# 日本語モデル
nlp = spacy.load('ja_core_news_lg')

In [2]:
import pandas as pd
import re

# 日本語フィードデータの読み込み、確認
feeds = pd.read_csv('data/output_jp.csv')
feeds.head()

,url,title,summary
0,https://www.nhk.or.jp/rss/news/cat0.xml,大阪 クリニック放火事件からまもなく1年 遺族が手記公開,大阪のビルでクリニックが放火され、巻き込まれた26人が亡くなった事件からまもなく1年となるの...
1,https://www.nhk.or.jp/rss/news/cat0.xml,ワールドカップ 日本 森保監督 今大会の成果と今後の課題は,サッカーのワールドカップカタール大会の決勝トーナメント1回戦で、クロアチアに敗れた日本代表の...
2,https://www.nhk.or.jp/rss/news/cat0.xml,ロシア空軍基地に“ウクライナ軍の無人機攻撃” 大きな打撃か,ロシア国内の複数の空軍基地で爆発があり、ロシア国防省はウクライナ軍の無人機による攻撃を受けた...
3,https://www.nhk.or.jp/rss/news/cat0.xml,サッカー日本代表 たどり着いたもう1つの“新しい景色”,11月20日に開幕したサッカーのワールドカップカタール大会。7回目の出場となった日本代表はこ...
4,https://www.nhk.or.jp/rss/news/cat0.xml,北朝鮮 日本海向け約100発砲撃 2日連続 “米韓両軍への警告”,韓国軍は、北朝鮮が6日、日本海に向けておよそ100発の砲撃を行ったと発表しました。5日もおよ...


In [3]:
# 1行目を確認
print('Title:', feeds.iloc[0].title)
print('Summary:', feeds.iloc[0].summary)

Title: 大阪 クリニック放火事件からまもなく1年 遺族が手記公開
Summary: 大阪のビルでクリニックが放火され、巻き込まれた26人が亡くなった事件からまもなく1年となるのを前に、被害者の遺族2人が報道陣に手記を寄せました。「いまだに現実を受け入れられていません」などと心情を記し、犯罪被害者の遺族への国の支援が十分でない実情を訴えています。


In [4]:
# title と summary を結合
# str.cat() により複数列の文字列を結合
# - sep=' ': 間に挟む文字列
# - na_rep='': NaN は空文字列に変換（指定しないと結合結果が NaN になる）
feeds['text'] = feeds['title'].str.cat(feeds['summary'], sep='。', na_rep='')

# 不要になった列を削除した処理用の DataFrame
df = feeds.drop(['title', 'summary'], axis=1)

# 確認
df.head()

,url,text
0,https://www.nhk.or.jp/rss/news/cat0.xml,大阪 クリニック放火事件からまもなく1年 遺族が手記公開。大阪のビルでクリニックが放火され、...
1,https://www.nhk.or.jp/rss/news/cat0.xml,ワールドカップ 日本 森保監督 今大会の成果と今後の課題は。サッカーのワールドカップカタール...
2,https://www.nhk.or.jp/rss/news/cat0.xml,ロシア空軍基地に“ウクライナ軍の無人機攻撃” 大きな打撃か。ロシア国内の複数の空軍基地で爆発...
3,https://www.nhk.or.jp/rss/news/cat0.xml,サッカー日本代表 たどり着いたもう1つの“新しい景色”。11月20日に開幕したサッカーのワー...
4,https://www.nhk.or.jp/rss/news/cat0.xml,北朝鮮 日本海向け約100発砲撃 2日連続 “米韓両軍への警告”。韓国軍は、北朝鮮が6日、日...


### 日本語テキストに対する前処理

- 表記の正規化
- トークン化（形態素解析）
- ストップワードの除去
- 見出し語化

In [5]:
# トークン化（形態素解析）
# - spaCy では表記の正規化も一緒に行われる

# 例として最初の行のテキストだけを処理
text = df['text'].iloc[0]

# 処理内容を確認
for w in nlp(text):
    t = w.text    # 単語
    p = w.pos_    # 品詞
    l = w.lemma_  # 原型
    s = w.is_stop # ストップワードの場合 True
    print('{}\t{}\t{}\t{}'.format(t, p, l, s))

大阪	PROPN	大阪	False
クリニック	NOUN	クリニック	False
放火	NOUN	放火	False
事件	NOUN	事件	False
から	ADP	から	True
ま	NOUN	ま	True
も	ADP	も	True
なく	ADJ	ない	True
1	NUM	1	False
年	NOUN	年	False
遺族	NOUN	遺族	False
が	ADP	が	True
手記	NOUN	手記	False
公開	NOUN	公開	False
。	PUNCT	。	False
大阪	PROPN	大阪	False
の	ADP	の	True
ビル	NOUN	ビル	False
で	ADP	で	True
クリニック	NOUN	クリニック	False
が	ADP	が	True
放火	VERB	放火	False
さ	AUX	する	True
れ	AUX	れる	True
、	PUNCT	、	False
巻き	VERB	巻く	False
込ま	VERB	込む	False
れ	AUX	れる	True
た	AUX	た	True
26	NUM	26	False
人	NOUN	人	False
が	ADP	が	True
亡くなっ	VERB	亡くなる	False
た	AUX	た	True
事件	NOUN	事件	False
から	ADP	から	True
ま	NOUN	ま	True
も	ADP	も	True
なく	ADJ	ない	True
1	NUM	1	False
年	NOUN	年	False
と	ADP	と	True
なる	VERB	なる	True
の	SCONJ	の	True
を	ADP	を	True
前	NOUN	前	False
に	ADP	に	True
、	PUNCT	、	False
被害	NOUN	被害	False
者	NOUN	者	False
の	ADP	の	True
遺族	NOUN	遺族	False
2人	NOUN	2人	False
が	ADP	が	True
報道	NOUN	報道	False
陣	NOUN	陣	False
に	ADP	に	True
手記	NOUN	手記	False
を	ADP	を	True
寄せ	VERB	寄せる	False
まし	AUX	ます	False
た	AUX	た	True
。	PUNCT	。	False
「	PUNCT	「	False
いまだ	ADV	いまだ	

In [6]:
# 不要な単語を除去
# - ストップワード (is_stop)
# - いくつかの品詞
#     AUX: 助動詞
#     PUNCT: 句読点
#     SPACE: 空白文字
#     SYM: 記号
#     X: その他
# - うまく取り除けない単語や文字
stop_pos = ['AUX', 'PUNCT', 'SPACE', 'SYM', 'X']
stop_words = ['.']

def token_to_add(w):
    t = w.text    # 単語
    p = w.pos_    # 品詞
    l = w.lemma_  # 原型

    # ストップワードは None を返す
    if w.is_stop:
        return None
    if p in stop_pos:
        return None
    if l in stop_words:
        return None

    if len(l) == 0:
        return t
    return l

# 処理を確認
tokens_wo_stop_words = []

for w in nlp(text):
    t = token_to_add(w)
    if t is not None:
        tokens_wo_stop_words.append(t)

tokens_wo_stop_words

['大阪',
 'クリニック',
 '放火',
 '事件',
 '1',
 '年',
 '遺族',
 '手記',
 '公開',
 '大阪',
 'ビル',
 'クリニック',
 '放火',
 '巻く',
 '込む',
 '26',
 '人',
 '亡くなる',
 '事件',
 '1',
 '年',
 '前',
 '被害',
 '者',
 '遺族',
 '2人',
 '報道',
 '陣',
 '手記',
 '寄せる',
 'いまだ',
 '現実',
 '受ける',
 '入れる',
 '心情',
 '記す',
 '犯罪',
 '被害',
 '者',
 '遺族',
 '国',
 '支援',
 '十分',
 '実情',
 '訴える']

### テキストのベクトル化

In [7]:
from sklearn.feature_extraction.text import CountVectorizer

# トークン化されたリストを結合
text_list = [' '.join(tokens_wo_stop_words)]

# 初期化
vectorizer = CountVectorizer()

# ベクトル化
vector = vectorizer.fit_transform(text_list)

# 確認
print(vector)
print(vectorizer.get_feature_names_out())

# ベクトルの単語との対応
for i in vector[0].indices:
    # print()
    # - end=' ': 改行の代わりに空白を出力
    print(vectorizer.get_feature_names_out()[i], end=' ')

  (0, 12)	2
  (0, 3)	2
  (0, 19)	2
  (0, 5)	2
  (0, 26)	3
  (0, 17)	2
  (0, 8)	1
  (0, 4)	1
  (0, 15)	1
  (0, 25)	1
  (0, 0)	1
  (0, 6)	1
  (0, 22)	2
  (0, 1)	1
  (0, 11)	1
  (0, 14)	1
  (0, 2)	1
  (0, 21)	1
  (0, 10)	1
  (0, 7)	1
  (0, 16)	1
  (0, 23)	1
  (0, 20)	1
  (0, 18)	1
  (0, 9)	1
  (0, 13)	1
  (0, 24)	1
['26' '2人' 'いまだ' 'クリニック' 'ビル' '事件' '亡くなる' '入れる' '公開' '十分' '受ける' '報道' '大阪'
 '実情' '寄せる' '巻く' '心情' '手記' '支援' '放火' '犯罪' '現実' '被害' '記す' '訴える' '込む' '遺族']
大阪 クリニック 放火 事件 遺族 手記 公開 ビル 巻く 込む 26 亡くなる 被害 2人 報道 寄せる いまだ 現実 受ける 入れる 心情 記す 犯罪 支援 十分 実情 訴える 